<a href="https://colab.research.google.com/github/ComponentSoftTeam/AI-110/blob/main/notebooks/2_Transformer_Demo_with_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inspired by code originally from Sinan Ozdemir's [notebook](https://github.com/sinanuozdemir/oreilly-transformers-video-series/blob/main/notebooks/7_NLG_with_GPT.ipynb).

## 2.1 Introduction to the tokenizer and embeddings of GPT-2

In [1]:
%%capture
%pip install datasets transformers bertviz

In [24]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel, nn
from bertviz import model_view

set_seed(42)

In [36]:
generator = pipeline('text-generation', model='gpt2')

generator("Hello, I'm here in Budapest and I", max_length=30, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm here in Budapest and I want to tell you that my dear little dear baby is going to live for many years to come.\n"},
 {'generated_text': "Hello, I'm here in Budapest and I'm happy to share with you that this will be my first visit to Europe after all. Europe is not"},
 {'generated_text': 'Hello, I\'m here in Budapest and I can finally finally say thanks to you all," he says in the Hungarian. "It\'s just a sad'}]

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [38]:
'Erno' in tokenizer.get_vocab()

False

In [51]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Hi there. I am Erno, I am your instructor.'))

['Hi',
 'Ġthere',
 '.',
 'ĠI',
 'Ġam',
 'ĠEr',
 'no',
 ',',
 'ĠI',
 'Ġam',
 'Ġyour',
 'Ġinstructor',
 '.']

In [15]:
tokenizer.encode('Hi there. I am Erno, I am your instructor.')

[17250, 612, 13, 314, 716, 5256, 3919, 11, 314, 716, 534, 21187, 13]

In [5]:
encoded = tokenizer.encode('Hi there. I am Erno, I am your instructor.', return_tensors='pt')

encoded

tensor([[17250,   612,    13,   314,   716,  5256,  3919,    11,   314,   716,
           534, 21187,    13]])

In [6]:
encoded.shape

torch.Size([1, 13])

In [7]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
# Get all of the model's parameters as a list of tuples.
named_params = list(model.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(named_params)))

print('==== Embedding Layer ====\n')
for name, params in named_params[0:2]:
    print(f"{name:<55} {str(tuple(params.size())):>12}")

print('\n==== Decoders ====\n')
for name, params in named_params[2:146]:
    print(f"{name:<55} {str(tuple(params.size())):>12}")


print('\n==== Output Layer ====\n')
for name, params in named_params[-2:]:
    print(f"{name:<55} {str(tuple(params.size())):>12}")

print('\n==== LM Head Layer ====\n')
print(f"{'lm_head':<55} {str(tuple(next(model.lm_head.parameters()).size())):>12}")

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50257, 768)
transformer.wpe.weight                                   (1024, 768)

==== Decoders ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072, 768)
tr

In [20]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

Number of params: 124,439,808


In [16]:
model.transformer.wte(encoded)

tensor([[[-0.0679, -0.1280,  0.0666,  ...,  0.0494, -0.0610, -0.0470],
         [-0.0806,  0.0413,  0.0576,  ..., -0.0095, -0.1874, -0.0539],
         [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
         ...,
         [-0.0756,  0.0461,  0.0550,  ..., -0.0826,  0.0872, -0.0208],
         [ 0.1415, -0.1637, -0.0499,  ...,  0.0580,  0.0695,  0.0071],
         [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963]]],
       grad_fn=<EmbeddingBackward0>)

In [17]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]))

tensor([[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
          2.8267e-02,  5.4490e-02],
        [ 2.3959e-02, -5.3792e-02, -9.4879e-02,  ...,  3.4170e-02,
          1.0172e-02, -1.5573e-04],
        [ 4.2161e-03, -8.4764e-02,  5.4515e-02,  ...,  1.9745e-02,
          1.9325e-02, -2.1424e-02],
        ...,
        [ 1.6006e-03,  6.2476e-03,  1.0040e-01,  ..., -4.6657e-03,
          9.3994e-04, -5.8468e-03],
        [-3.5615e-03,  1.7494e-02,  1.0676e-01,  ..., -5.4367e-03,
         -7.9653e-04, -5.6959e-03],
        [ 5.9564e-05,  1.7205e-02,  9.6934e-02,  ..., -1.5799e-03,
         -8.6813e-04, -7.8220e-03]], grad_fn=<EmbeddingBackward0>)

In [34]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12]).reshape(1, 13))
initial_input


tensor([[[-0.0867, -0.3254,  0.0706,  ...,  0.0063, -0.0328,  0.0075],
         [-0.0566, -0.0125, -0.0373,  ...,  0.0247, -0.1772, -0.0540],
         [ 0.0509, -0.0961,  0.0828,  ..., -0.0538,  0.0689,  0.0749],
         ...,
         [-0.0740,  0.0523,  0.1554,  ..., -0.0873,  0.0881, -0.0266],
         [ 0.1379, -0.1462,  0.0569,  ...,  0.0525,  0.0687,  0.0014],
         [ 0.0467,  0.0059,  0.1253,  ..., -0.0751,  0.0488,  0.0885]]],
       grad_fn=<AddBackward0>)

In [48]:
contextful_embedding == model(encoded, output_hidden_states=True).hidden_states[-1]
print(contextful_embedding.shape)
contextful_embedding

torch.Size([1, 13, 768])


tensor([[[-5.1051e-02, -1.9038e-01, -3.3829e-01,  ..., -1.9444e-01,
          -4.5692e-02, -1.8426e-01],
         [ 3.4731e-01, -1.0124e-03, -2.9825e-01,  ...,  1.7716e-01,
           1.9853e-01,  4.4778e-01],
         [ 3.2489e-01, -7.4511e-02, -3.0923e-01,  ..., -1.1854e-01,
          -1.7494e-01, -2.1527e-02],
         ...,
         [ 1.4272e-01, -1.4377e-01,  2.1839e-01,  ...,  4.7257e-02,
           4.5007e-01,  2.0302e-01],
         [ 7.8564e-02, -1.3801e-01, -1.5103e+00,  ...,  4.3329e-02,
           1.2174e-01,  1.5725e-01],
         [ 2.0761e-01, -3.0169e-01, -3.8818e-01,  ...,  1.0580e-01,
          -4.1715e-02,  2.2291e-01]]], grad_fn=<NativeLayerNormBackward0>)

In [40]:
module_input = initial_input
for module in model.transformer.h:
    module_input = module(module_input)[0]

print(f"\nfinal module output before normalization:")
print(initial_input.shape)
print(initial_input)

contextful_embedding = model.transformer.ln_f(initial_input)
print(f"\ncontextful_embedding:")
print(contextful_embedding.shape)
print(contextful_embedding)

logits = model.lm_head(contextful_embedding)
print(f"\nlogits:")
print(logits.shape)
print(logits)

probabilities = nn.functional.softmax(logits, dim=2)
print(f"\nprobabilities:")
print(probabilities.shape)
print(probabilities)


final module output before normalization:
torch.Size([1, 13, 768])
tensor([[[ 1.2110e-02, -1.6717e+00, -1.3894e+00,  ..., -1.6987e+00,
           2.6213e-01, -1.8849e+00],
         [ 3.6915e+00, -8.3287e-01, -2.3966e+00,  ...,  2.4070e+00,
           2.9721e+00,  5.2475e+00],
         [ 4.5242e+00, -1.7653e+00, -2.7203e+00,  ..., -2.1181e+00,
          -2.8785e+00, -1.0734e+00],
         ...,
         [ 1.6042e+00, -2.3082e+00,  2.4460e+00,  ...,  7.7297e-01,
           7.1875e+00,  2.2483e+00],
         [ 6.2690e-01, -2.3633e+00, -1.2819e+01,  ...,  5.1152e-01,
           1.9169e+00,  1.3856e+00],
         [ 2.7683e+00, -4.8010e+00, -3.3416e+00,  ...,  1.9459e+00,
          -3.5617e-01,  2.9281e+00]]], grad_fn=<AddBackward0>)

contextful_embedding:
torch.Size([1, 13, 768])
tensor([[[-5.1051e-02, -1.9038e-01, -3.3829e-01,  ..., -1.9444e-01,
          -4.5692e-02, -1.8426e-01],
         [ 3.4731e-01, -1.0124e-03, -2.9825e-01,  ...,  1.7716e-01,
           1.9853e-01,  4.4778e-01],
    

In [38]:
print('Original words:')
print(tokenizer.convert_ids_to_tokens(tokenizer.encode('Hi there. I am Erno, I am your instructor.', return_tensors='pt')[0]))
print('\nMost probable next words:')
print(tokenizer.convert_ids_to_tokens(probabilities.argmax(dim=2)[0]))

Original words:
['Hi', 'Ġthere', '.', 'ĠI', 'Ġam', 'ĠEr', 'no', ',', 'ĠI', 'Ġam', 'Ġyour', 'Ġinstructor', '.']

Most probable next words:
['.', ',', 'ĠI', "'m", 'Ġnot', 'nie', '.', 'Ġand', 'Ġam', 'Ġthe', 'Ġfather', '.', 'ĠI']


## 2.2 Introduction to GPT-2 masked multi-headed attention

In [52]:
import torch
import pandas as pd


In [53]:
phrase = 'My friend was right about this book. It is really funny.' ###
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

12

In [54]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1492,   13,  632,  318, 1107, 8258,
           13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [55]:
response.attentions[-1].shape  # From the final decoder

torch.Size([1, 12, 13, 13])

In [56]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [57]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0]) ###

tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġbook',
 '.',
 'ĠIt',
 'Ġis',
 'Ġreally',
 'Ġfunny',
 '.']

In [58]:
### Layer index 9, head 0. Check out the 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġbook,.,ĠIt,Ġis,Ġreally,Ġfunny,.
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġbook,0.956,0.004,0.002,0.002,0.004,0.004,0.028,0.000,0.000,0.000,0.000,0.000,0.000
.,0.728,0.010,0.003,0.003,0.002,0.008,0.235,0.011,0.000,0.000,0.000,0.000,0.000
ĠIt,0.310,0.002,0.003,0.006,0.012,0.020,0.601,0.014,0.031,0.000,0.000,0.000,0.000
Ġis,0.344,0.005,0.002,0.004,0.004,0.016,0.517,0.011,0.067,0.030,0.000,0.000,0.000


In [59]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [60]:
tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġbook',
 '.',
 'ĠIt',
 'Ġis',
 'Ġreally',
 'Ġfunny',
 '.']

In [61]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [62]:
response.logits.shape

torch.Size([1, 13, 50257])

In [63]:
states = torch.nn.functional.softmax(response.logits, dim=2)
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0]), [f"{states[0][i][j.item()].item():.2%}" for i, j in enumerate(states.argmax(2)[0])]),
    columns=['Sequence up until', 'Next token with highest probability', 'Probability']
)

,Sequence up until,Next token with highest probability,Probability
0,My,Ċ,1.32%
1,Ġfriend,",",13.32%
2,Ġwas,Ġa,4.96%
3,Ġright,.,23.58%
4,Ġabout,Ġthat,20.61%
5,Ġthis,.,22.32%
6,Ġbook,.,27.37%
7,.,ĠI,19.27%
8,ĠIt,'s,37.43%
9,Ġis,Ġa,17.66%


In [64]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this book. It is really funny. I think it is a great book'}]

In [65]:
generator(phrase, max_length=20, num_return_sequences=3, do_sample=True)  # greedy search with sampling

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this book. It is really funny. It is the first person you have'},
 {'generated_text': 'My friend was right about this book. It is really funny. He wrote these characters. He says'},
 {'generated_text': "My friend was right about this book. It is really funny. That's what this book is about"}]